https://www.ncbi.nlm.nih.gov/genome/guide/human/#download

**Set environment**

In [1]:
suppressWarnings(suppressMessages(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(FD_RES, "source", "annotation_gene")
print(dir(fdiry))

[1] "GRCh38_latest_genomic.20230315.gff.gz"       
[2] "K562_TSS_selected_by_higest_Pol2_TSS_sig.out"
[3] "Readme.md"                                   


In [3]:
fdiry = file.path(FD_RES, "source", "annotation_gene")
fname = "GRCh38_latest_genomic.20230315.gff.gz"
fpath = file.path(fdiry, fname)

txt = paste("zcat", fpath, "|", "head -n 10")
res = system(txt, intern = TRUE)
for (x in res){cat(x, "\n")}

##gff-version 3 
#!gff-spec-version 1.21 
#!processor NCBI annotwriter 
#!genome-build GRCh38.p14 
#!genome-build-accession NCBI_Assembly:GCF_000001405.40 
#!annotation-date 03/15/2023 
#!annotation-source NCBI RefSeq GCF_000001405.40-RS_2023_03 
##sequence-region NC_000001.11 1 248956422 
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606 
NC_000001.11	RefSeq	region	1	248956422	.	+	.	ID=NC_000001.11:1..248956422;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=chromosome;mol_type=genomic DNA 


## Import data

In [4]:
fdiry = file.path(FD_RES, "source", "RNAseq")
fname = "K562.ENCSR615EEK.ENCFF421TJX.RNAseq.filtered.mapped.tsv"
fpath = file.path(fdiry, fname)

### read file
dat = read_tsv(fpath)

### assign and show
dat_rnaseq_gene_filter = dat
print(dim(dat))
head(dat)

Rows: 30925 Columns: 7
── Column specification ──────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): Gene_Stable_ID, Gene_Stable_ID_Version, Gene_Name, Gene_Synonym, Tr...
dbl (2): Length, TPM

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 30925     7


Gene_Stable_ID,Gene_Stable_ID_Version,Gene_Name,Gene_Synonym,Transcript_Stable_ID_Version,Length,TPM
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
ENSG00000125863,ENSG00000125863.19,MKKS,BBS6,"ENST00000347364.7,ENST00000399054.6",2710.75,4.04
ENSG00000285508,ENSG00000285508.1,MKKS,BBS6,ENST00000609375.1,882.00,5.05
ENSG00000285723,ENSG00000285723.1,MKKS,BBS6,ENST00000649912.1,192.00,21.39
ENSG00000125863,ENSG00000125863.19,MKKS,HMCS,"ENST00000347364.7,ENST00000399054.6",2710.75,4.04
ENSG00000285508,ENSG00000285508.1,MKKS,HMCS,ENST00000609375.1,882.00,5.05
ENSG00000285723,ENSG00000285723.1,MKKS,HMCS,ENST00000649912.1,192.00,21.39


In [5]:
### set file directory
fdiry = file.path(FD_RES, "source", "annotation_gene")
fname = "K562_TSS_selected_by_higest_Pol2_TSS_sig.out"
fpath = file.path(fdiry, fname)

### read file
dat = read_tsv(fpath)

### assign and show
dat_gene_tss_pol2 = dat
print(dim(dat))
head(dat)

Rows: 29330 Columns: 4
── Column specification ──────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): gene_name, chr
dbl (2): TSS, TSS_POL2_sig

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 29330     4


gene_name,chr,TSS,TSS_POL2_sig
<chr>,<chr>,<dbl>,<dbl>
A1BG-AS1,chr19,58351969,1.52180000
A1BG,chr19,58353492,5.62284000
A1CF,chr10,50885675,0.00706126
A2M-AS1,chr12,9065176,0.00201544
A2M,chr12,9115919,0.04005220
A2ML1,chr12,8845003,0.26862300


## Construct a bed file

In [7]:
dat = dat_gene_tss_pol2
dat = dat %>% 
    dplyr::rename(
        "Chrom" = "chr",
        "Name"  = "gene_name",
        "Score" = "TSS_POL2_sig",
        "Start" = "TSS"
    ) %>%
    dplyr::mutate(End = Start + 1) %>%
    dplyr::select(Chrom, Start, End, Name, Score) %>%
    dplyr::arrange(Chrom, Start, End)

dat_tss = dat
print(dim(dat))
head(dat)

[1] 29330     5


Chrom,Start,End,Name,Score
<chr>,<dbl>,<dbl>,<chr>,<dbl>
chr1,11873,11874,DDX11L1,0.00023
chr1,17436,17437,MIR6859-1,9.43812
chr1,17436,17437,MIR6859-2,9.43812
chr1,17436,17437,MIR6859-3,9.43812
chr1,17436,17437,MIR6859-4,9.43812
chr1,29370,29371,WASH7P,0.00023


In [8]:
vec = c(
    dat_rnaseq_gene_filter$Gene_Name,
    dat_rnaseq_gene_filter$Gene_Synonym)

dat = dat_tss
dat = dat %>%
    dplyr::filter(Name %in% vec)

dat_tss_filter = dat
print(dim(dat))
head(dat)

[1] 11899     5


Chrom,Start,End,Name,Score
<chr>,<dbl>,<dbl>,<chr>,<dbl>
chr1,29370,29371,WASH7P,2.30000e-04
chr1,827522,827523,LINC00115,6.44656e+01
chr1,827590,827591,LINC01128,6.44603e+01
chr1,876802,876803,FAM41C,7.88399e-03
chr1,959256,959257,NOC2L,1.04866e+02
chr1,960583,960584,KLHL17,8.22571e+00


## Explore

In [15]:
dat = dat_tss
dat = dat %>% dplyr::select(Chrom, Start, End) %>% dplyr::distinct()
print(nrow(dat))

dat = dat_tss_filter
dat = dat %>% dplyr::select(Chrom, Start, End) %>% dplyr::distinct()
print(nrow(dat))

[1] 28563
[1] 11830


In [ ]:
vec1 = dat_gene_tss_pol2$gene_name
vec1 = unique(vec1)
print(length(vec1))

vec2 = c(
    dat_rnaseq_gene_filter$Gene_Name,
    dat_rnaseq_gene_filter$Gene_Synonym)
vec2 = unique(vec2)
print(length(vec2))

vec3 = intersect(vec1, vec2)
print(length(vec3))

## create data dictionary

In [9]:
dat_desc = tibble::tribble(
    ~Name, ~Description,
    "Chrom",   "Chromosome",
    "Start",   "TSS position",
    "End",     "TSS position",
    "Name",    "Gene name",
    "Score",   "highest level of pol2 chip seq (ENCFF914WIS.bigWig) at [TSS-500, TSS+500] among TSS isoform"
)

## Explore

In [10]:
dat = dat_gene_tss_pol2
dat = dat %>% dplyr::filter(gene_name == "HBE1")
dat

gene_name,chr,TSS,TSS_POL2_sig
<chr>,<chr>,<dbl>,<dbl>
HBE1,chr11,5269945,319.925


## Save results

In [11]:
fdiry = file.path(FD_RES, "results", "region")
dir(fdiry)

[1] "annotation_ccre"                        
 [2] "annotation_chipseq_histone"             
 [3] "annotation_chipseq_tf"                  
 [4] "annotation_chipseq_tf_subset"           
 [5] "annotation_chromHMM"                    
 [6] "annotation_crispri_e2g"                 
 [7] "annotation_crispri_growth"              
 [8] "annotation_crispri_hcrff"               
 [9] "annotation_enhancer_junke"              
[10] "annotation_enhancer_merge"              
[11] "annotation_enhancer_zscore_junke"       
[12] "annotation_gencode"                     
[13] "annotation_tss_pol2"                    
[14] "hic_intact_deep"                        
[15] "hic_intact_ENCSR479XDG"                 
[16] "KS91_K562_ASTARRseq_peak_macs_input"    
[17] "region_screened_crispri_flowfish.bed"   
[18] "region_screened_selected_perbase.bed"   
[19] "region_screened_selected_perbase.bed.gz"
[20] "region_screened_selected.bed"           
[21] "region_screened_selected.bed.gz"        
[22] "region_screened_tmpra_crispr.bed"       
[23] "region_screened_tmpra.bed"              
[24] "z_annotation_hic"                       
[25] "z_annotation_inactive"                  
[26] "z_annotation_tss_pol2_span"

In [12]:
fdiry = file.path(FD_RES, "results", "region", "annotation_tss_pol2")
fname = "K562.TSS.selected_by_highest_Pol2_signal.bed.gz"
fpath = file.path(fdiry, fname)

dat = dat_tss
write_tsv(dat, fpath, col_names = FALSE)

In [13]:
fdiry = file.path(FD_RES, "results", "region", "annotation_tss_pol2")
fname = "K562.TSS.selected_by_highest_Pol2_signal.filtered_by_RNAseq_TPM.bed.gz"
fpath = file.path(fdiry, fname)

dat = dat_tss_filter
write_tsv(dat, fpath, col_names = FALSE)

In [14]:
fdiry = file.path(FD_RES, "results", "region", "annotation_tss_pol2")
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = dat_desc
write_tsv(dat, fpath)